# Finding Pauli Matrices from Hamiltonian

I am following the algorithm described here: https://arxiv.org/abs/1907.01493

In [1]:
import numpy as np

### First we need write down some definitions

In [285]:
t0 = np.array([[1,0],[0,0]])
t1 = np.array([[0,1],[0,0]])
t2 = np.array([[0,0],[1,0]])
t3 = np.array([[0,0],[0,1]])
T = [t0,t1,t2,t3]

s0 = np.array([[1,0],[0,1]])
s1 = np.array([[0,1],[1,0]])
s2 = np.array([[0,1],[-1,0]])
s3 = np.array([[1,0],[0,-1]])
S = [s0,s1,s2,s3]

In [286]:
def Mkron(Mlist):
    Mno=len(Mlist)
    mtp=Mlist[0]
    for i in range(1,Mno):
        mtp=np.kron(mtp,Mlist[i])
    return mtp

# A function to print out the binary number
def bi(num):
    bi = bin(num)
    out = ""
    for i in range(2,len(bi)):
        out = out + bi[i]
    return out

### Create an arbitrary test Hamiltonian so that I can check each step.

In [287]:
H = 0.2*Mkron([t2,t3,t0]) + 0.6*Mkron([t0,t3,t3]) + 0.9*Mkron([t2,t1,t1]) + 1.1*Mkron([t1,t2,t0])

H

array([[0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 1.1, 0. , 0. , 0. ],
       [0. , 0. , 0. , 0.6, 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0.9, 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0.2, 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ]])

## Create c0

The first step is collect the elements of the Hamiltonian so they can be called based by the qubit they on which they act and an index which charecterizes the single qubit subspace

In [280]:
def c0(H):
    #Get the size of H and number of qubits.  
    #I have assumed that log_2 N is an integer
    N = len(H)
    Q = int(np.log2(N))
    
    #Create a skeleten for c
    c = {}
    
    #fill c with elements of H
    for i in range(N):
        for j in range(N):
            rQ = ''
            for q in range(Q):
                iq = np.mod(int((i)/2**q),2)
                jq = np.mod(int((j)/2**q),2)
                rQ = rQ + str(2*iq + jq)
            c[rQ] = H[i][j]
                
    return c


### Let us quickly check that we collected the terms correctly

In [284]:
N = len(H)
Q = int(np.log2(N))

c = c0(H)

Htst = 0*H
for rQ in c.keys():
    Htst += c[rQ]*Mkron([T[int(rQ[2])],T[int(rQ[1])],T[int(rQ[0])]])

np.amax(np.abs(Htst-H))

0.0

## Find the weights of the Pauli Matrices

In order to find the wieghts of the Pauli matrices, we will break the problem up into the individual qubits.  This is possible becuase the transformation is seperable.  

In [288]:
# The single qubit transformation
m = np.array([[1,0,0,1],[0,1,1,0],[0,1,-1,0],[1,0,0,-1]])

#Define the step forward for cq
def cb(ca,q):
    cout = {}
    for key in ca.keys():
        rQ = key
        rQl = list(rQ)
        rqq = int(rQl[q])
        cQrQ = 0
        for rq in range(4):
                rQl[q] = rq
                rQ = ''.join(str(rq) for rq in rQl)
                cQrQ += 1/2 * m[rqq,rq]*ca[rQ]
        cout[key] = cQrQ
    return cout

In [289]:
#Define the final cQ
def find_cQ(c0):
    Q = len(list(c0.keys())[0])
    cq = c0
    for q in range(Q):
        cq = cb(cq,q)
    return cq

### Check that it has worked

In [291]:
cQ = find_cQ(c)

Htst = 0*H
for rQ in cQ.keys():
    Htst += cQ[rQ]*Mkron([S[int(rQ[2])],S[int(rQ[1])],S[int(rQ[0])]])
    
np.amax(np.abs(H-Htst))

2.220446049250313e-16

In [292]:
pwd

'C:\\Users\\jsten\\IBMQ\\Hubbard_symmetries'